In [23]:
# Hello, TensorFlow!
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


In [24]:
# Download MNIST datasource
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [25]:
# 通过操作符号变量来描述这些可交互的操作单元
x = tf.placeholder(tf.float32, [None, 784])

In [26]:
# 权重值和偏置量
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [27]:
# softmax回归模型
y = tf.nn.softmax(tf.matmul(x,W) + b)

In [28]:
# 交叉熵
y_ = tf.placeholder("float", [None,10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [29]:
# 优化算法
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)

In [30]:
# 初始化
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [31]:
# 训练 1000次
for _ in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [32]:
# 评估模型
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9041
